<a href="https://colab.research.google.com/github/Abhinav-Jindal05/freecodecamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,TextVectorization,Embedding,GlobalAveragePooling1D,Dropout
import pandas as pd
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
train_df['label'].value_counts()
test_df['label'].value_counts()

In [ ]:
x_train = np.array(train_df['message'])
y_train = np.array(train_df['label'])
x_test = np.array(test_df['message'])
y_test = np.array(test_df['label'])

In [ ]:
y_train = np.where(y_train == 'spam', 1, 0)
y_test = np.where(y_test == 'spam', 1, 0)

In [ ]:
vectorizer = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=171,
)
vectorized = vectorizer.adapt(tf.constant(x_train))

In [ ]:
# Create a frozen vectorizer layer to reuse inside loop
frozen_vectorizer = tf.keras.models.Sequential([vectorizer])
frozen_vectorizer.trainable = False


In [ ]:
weights = []
train_errors = []

for i in range(10):
    model = Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),
        frozen_vectorizer,
        Embedding(1000, 64),
        GlobalAveragePooling1D(),
        Dense(128, activation='relu', name='layer2'),
        Dense(64, activation='relu', name='layer3'),
        Dense(1, activation='linear', name='layer4')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(
        x_train, y_train,
        epochs=20,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)],
        verbose=0
    )

    yhat = model.predict(x_train)
    yhat = tf.math.sigmoid(yhat)
    yhat = tf.where(yhat > 0.5, 1, 0)
    train_error = np.mean(yhat.numpy().flatten() != y_train)
    train_errors.append(train_error)

    weights.append(model.get_weights())

print(train_errors)



In [ ]:
# Rebuild the exact same model architecture
best_model = Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    frozen_vectorizer,
    Embedding(1000, 64),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu', name='layer2'),
    Dense(64, activation='relu', name='layer3'),
    Dense(1, activation='linear', name='layer4')
])

# Load the best weights
best_index = train_errors.index(min(train_errors))
best_model.set_weights(weights[best_index])

# Re-evaluate
yhat = best_model.predict(x_train)
yhat = tf.math.sigmoid(yhat)
yhat = tf.where(yhat > 0.5, 1, 0)
train_error = np.mean(yhat.numpy().flatten() != y_train)
print("Best train error after restoring full model:", train_error)


In [ ]:
yhat = best_model.predict(x_test)
yhat = tf.math.sigmoid(yhat)
yhat = tf.where(yhat > 0.5, 1, 0)
test_error = np.mean(yhat.numpy().flatten() != y_test)
print(test_error)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(tf.constant([pred_text]))
  prob = tf.math.sigmoid(prediction)[0][0]
  label = "ham" if prob < 0.5 else "spam"
  return [prob.numpy(), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
